In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from random import randint
import time
from IPython.core.display import clear_output
from warnings import warn
warn("Warning Simulation")

/Users/nathanorenstein/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Warning Simulation
  if __name__ == '__main__':


In [2]:
starts = [str(i) for i in range(1, 2000, 50)]

In [3]:
# Each list stores different attributes of the data
names = []
years = []
imdb_ratings = []
metascores = []
votes = []
runtimes = []
genres = []

# Will monitor the loop
start_time = time.time()
requests = 0

# Starting at movie 1, then 51, 101, etc.
for start in starts: 
    
    # Make a get request
    response = get('https://www.imdb.com/search/title/?title_type=feature&num_votes=40000,&genres=drama&'
                   + 'sort=user_rating,desc&start=' + start + '&ref_=adv_nxt')
    
    
    # Pause the loop
    sleep(randint(8, 15))
    
    # Monitor the requests
    requests += 1

    if requests > 41:
        warn('Number of requests was greater than expected.')
        break
        
    # Parse the content of the request with BeautifulSoup
    page_html = BeautifulSoup(response.text, 'html.parser')
    
    # Select all the 50 movies containers from a single page
    mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
    
    # For every movie of these 50
    for container in mv_containers:
        
        # Scrape the name
        name = container.h3.a.text
        names.append(name)
        
        # Scrape the year
        year = container.h3.find('span', class_ = 'lister-item-year').text
        years.append(year)
        
        # Scrape the IMDb rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)
        
        if container.find('div', class_ = 'ratings-metascore') is not None:
            m_score = int(container.find('span', class_ = 'metascore').text)
        else:
            m_score = None
        metascores.append(m_score)
        
        #Scrape the number of votes
        vote = container.find('span', attrs = {'name':'nv'})['data-value']
        votes.append(int(vote))
        
        runtime = container.p.find('span', class_ = 'runtime').text
        runtimes.append(int(runtime.split()[0]))
        
        genre = container.p.find('span', class_ = 'genre').text
        whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')
        genre = ''.join(filter(whitelist.__contains__, genre))
        genres.append(genre.split())

In [4]:
movie_ratings = pd.DataFrame({'movie': names,
                             'year': years,
                             'imdb': imdb_ratings,
                             'metascore': metascores,
                             'votes': votes,
                             'runtime': runtime,
                             'genres': genres})
print(movie_ratings.info())
movie_ratings.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 7 columns):
movie        2000 non-null object
year         2000 non-null object
imdb         2000 non-null float64
metascore    1868 non-null float64
votes        2000 non-null int64
runtime      2000 non-null object
genres       2000 non-null object
dtypes: float64(2), int64(1), object(4)
memory usage: 109.5+ KB
None


,movie,year,imdb,metascore,votes,runtime,genres
0,The Shawshank Redemption,(1994),9.3,80.0,2289500,155 min,[Drama]
1,The Godfather,(1972),9.2,100.0,1580220,155 min,"[Crime, Drama]"
2,The Dark Knight,(2008),9.0,84.0,2254571,155 min,"[Action, Crime, Drama]"
3,The Godfather: Part II,(1974),9.0,90.0,1104034,155 min,"[Crime, Drama]"
4,The Lord of the Rings: The Return of the King,(2003),8.9,94.0,1610555,155 min,"[Action, Adventure, Drama]"


In [5]:
movie_ratings

,movie,year,imdb,metascore,votes,runtime,genres
0,The Shawshank Redemption,(1994),9.3,80.0,2289500,155 min,[Drama]
1,The Godfather,(1972),9.2,100.0,1580220,155 min,"[Crime, Drama]"
2,The Dark Knight,(2008),9.0,84.0,2254571,155 min,"[Action, Crime, Drama]"
3,The Godfather: Part II,(1974),9.0,90.0,1104034,155 min,"[Crime, Drama]"
4,The Lord of the Rings: The Return of the King,(2003),8.9,94.0,1610555,155 min,"[Action, Adventure, Drama]"
5,Pulp Fiction,(1994),8.9,94.0,1788087,155 min,"[Crime, Drama]"
6,Schindler's List,(1993),8.9,94.0,1188650,155 min,"[Biography, Drama, History]"
7,12 Angry Men,(1957),8.9,96.0,672450,155 min,"[Crime, Drama]"
8,The Mountain II,(2016),8.8,NaN,104113,155 min,"[Action, Drama, War]"
9,Fight Club,(1999),8.8,66.0,1815551,155 min,[Drama]
